# 2020 1C 3ra Oportunidad

(***) Un equipo del Ministerio de Salud de la Nación se encuentra realizando distintos análisis de datos a partir de la información que tienen unificada a nivel nacional de los sistemas de información para el seguimiento de la campaña de vacunación contra el sars-cov-2.

Es por ello que cuentan con información en los siguientes archivos csv:

```
centro_de_vacunacion.csv (id_centro_vacunacion, nombre, direccion, codigo_postal, partido, provincia)
```

```
categoria_plan_vacunacion.csv  (id_categoria, nombre_categoria, prioridad)
```

```
aplicaciones.csv (id_ciudadano, fecha_aplicacion, id_categoria, id_centro_vacunacion, id_unico_dosis_vacuna, id_tipo_vacuna)
```

El primer archivo cuenta con información de los distintos centros de vacunación a lo largo y ancho del país donde se realizan aplicaciones de las vacunas disponibles. Este archivo nos da información geográfica de donde se encuentran los centros de vacunación en cada provincia.

El segundo tiene información de las distintas categorías que tiene el plan de vacunación, y la prioridad de las mismas, las cuales son valores numéricos siendo el valor 1 el valor más alto de prioridad. Por ejemplo, la categoría “Personal Esencial: Salud” tiene prioridad 1. Tener en cuenta que múltiples categorías pueden tener el mismo número de prioridad.

En el último archivo, contamos con la información de ciudadanos que se fueron realizando aplicaciones dentro del plan de vacunación, cuando se aplicó la misma (en formato “YYYY-mm-dd”), en que lugar y bajo qué categoría se hizo. Algo a considerar aquí es que también contamos con información específica de la dosis que se le aplicó a la personas y el id del tipo de vacuna.

Algo importante a considerar es que en la Argentina solamente se cuenta por el momento con vacunas que requieren 2 dosis para tener inmunidad completa, es por eso que en aplicaciones.csv pueden haber múltiples entradas por ciudadano.

Se desea:

a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).

b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.



In [23]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [24]:
# generamos valores aleatorio a utilizar en la generacion de los data frames
id_ciudadano = np.random.randint(1, high = 10, size = 20)
id_centro_vacunacion = rd.sample(range(1,21),20)
id_categoria = rd.sample(range(1,21),20)
id_provincia = np.random.randint(1, high = 5, size = 20)


In [25]:
# armamos el df de centros de vacunacion

centros_data = {
        'id_centro_vacunacion': id_centro_vacunacion,
        'nombre': id_centro_vacunacion,
        'direccion': id_centro_vacunacion,
        'codigo_postal': id_centro_vacunacion,
        'partido': id_centro_vacunacion,
        'provincia': id_provincia, 
}

df_centros = pd.DataFrame(centros_data, columns = centros_data.keys())

In [26]:
# armamos el data frame de datos de productos
categoria_data = {
        'id_categoria': id_categoria,
        'nombre_categoria': id_categoria,
        'prioridad': np.random.randint(1, high=5, size=20),
}

df_categoria = pd.DataFrame(categoria_data, columns = categoria_data.keys())

In [27]:
aplicaciones_data = {
        'id_ciudadano': id_ciudadano,
        'fecha': pd.date_range(start = '2020-11-01', end='2021-04-01', periods=20),
        'id_categoria': id_categoria,
        'id_centro_vacunacion': id_centro_vacunacion,
        'id_unico_dosis_vacuna': rd.sample(range(1,199), 20),
        'id_tipo_vacuna': np.random.randint(1,high=3, size=20), 
}

df_aplicaciones = pd.DataFrame(aplicaciones_data, columns = aplicaciones_data.keys())

In [28]:
df_centros.head()

,id_centro_vacunacion,nombre,direccion,codigo_postal,partido,provincia
0,11,11,11,11,11,3
1,1,1,1,1,1,2
2,6,6,6,6,6,4
3,16,16,16,16,16,2
4,20,20,20,20,20,2


In [29]:
df_categoria.head()

,id_categoria,nombre_categoria,prioridad
0,2,2,3
1,4,4,1
2,13,13,4
3,15,15,2
4,10,10,1


In [30]:
df_aplicaciones.head()

,id_ciudadano,fecha,id_categoria,id_centro_vacunacion,id_unico_dosis_vacuna,id_tipo_vacuna
0,3,2020-11-01 00:00:00.000000000,2,11,136,2
1,9,2020-11-08 22:44:12.631578947,4,1,157,2
2,5,2020-11-16 21:28:25.263157894,13,6,179,2
3,8,2020-11-24 20:12:37.894736842,15,16,52,2
4,3,2020-12-02 18:56:50.526315789,10,20,97,1


a) Indicar el porcentaje de ciudadanos vacunados en prioridad 1 que tienen inmunidad completa (recibieron dos aplicaciones de vacuna).

In [31]:
categ_prioridad_1 = df_categoria.loc[df_categoria['prioridad'] == 1, ['id_categoria']]
categ_prioridad_1

,id_categoria
1,4
4,10
11,20
17,16
18,12


In [42]:
vacs_prior_1 = pd.merge(df_aplicaciones,categ_prioridad_1,on='id_categoria',how='inner')
vacs_prior_1 = vacs_prior_1.loc[:,['id_ciudadano']]
vacs_prior_1['vacunas_aplicadas'] = 1

In [43]:
vacs_por_persona = vacs_prior_1.groupby('id_ciudadano').count().reset_index()
vacs_por_persona

,id_ciudadano,vacunas_aplicadas
0,3,1
1,5,1
2,6,2
3,9,1


In [46]:
cant_vacunados_2_veces = len(vacs_por_persona.loc[vacs_por_persona['vacunas_aplicadas']>=2])
cant_vacunados_2_veces

1

In [48]:
print(f"Porcentaje de personas que recibieron las 2 dosis: {cant_vacunados_2_veces/len(vacs_por_persona)*100}")

Porcentaje de personas que recibieron las 2 dosis: 25.0


b) Indicar cual es el top 5 de provincias que han inmunizado de forma completa a mayor cantidad de ciudadanos (es decir realizando dos aplicaciones de la vacuna) en el periodo temporal de diciembre de 2020 a marzo (inclusive) de 2021.

In [56]:
ultimos_meses = (df_aplicaciones['fecha']>=pd.to_datetime('2020-12-01')) & (df_aplicaciones['fecha']<pd.to_datetime('2021-04-01'))
vacs_recientes = df_aplicaciones[ultimos_meses]
vacs_recientes

,id_ciudadano,fecha,id_categoria,id_centro_vacunacion,id_unico_dosis_vacuna,id_tipo_vacuna
4,3,2020-12-02 18:56:50.526315789,10,20,97,1
5,2,2020-12-10 17:41:03.157894737,1,9,48,2
6,3,2020-12-18 16:25:15.789473684,8,15,33,1
7,9,2020-12-26 15:09:28.421052632,5,3,119,1
8,5,2021-01-03 13:53:41.052631579,9,19,75,2
9,5,2021-01-11 12:37:53.684210526,3,7,167,1
10,6,2021-01-19 11:22:06.315789474,7,12,131,2
11,6,2021-01-27 10:06:18.947368421,20,8,183,2
12,6,2021-02-04 08:50:31.578947368,14,5,79,1
13,9,2021-02-12 07:34:44.210526316,11,14,21,1


In [91]:
vacs_recientes['vacunas_aplicadas'] = 1
vacunados_completos = vacs_recientes.groupby('id_ciudadano').count()
vacunados_completos = vacunados_completos[vacunados_completos['vacunas_aplicadas']>1]
vacunados_completos = vacunados_completos.reset_index()
vacunados_completos

,id_ciudadano,id_centro_vacunacion,vacunas_aplicadas
0,2,2,2
1,3,3,3
2,5,3,3
3,6,4,4
4,9,2,2


In [96]:
vacs_ciudades = pd.merge(vacunados_completos,df_centros,on='id_centro_vacunacion',how='left')
vacs_ciudades = vacs_ciudades.loc[:,['provincia','vacunas_aplicadas']]
vacs_ciudades

,id_ciudadano,id_centro_vacunacion,vacunas_aplicadas,nombre,direccion,codigo_postal,partido,provincia
0,2,2,2,2,2,2,2,3
1,3,3,3,3,3,3,3,4
2,5,3,3,3,3,3,3,4
3,6,4,4,4,4,4,4,2
4,9,2,2,2,2,2,2,3


In [93]:
vacs_ciudades = vacs_ciudades.groupby('provincia').agg(sum)
vacs_ciudades = vacs_ciudades.reset_index()
vacs_ciudades.nlargest(5,'vacunas_aplicadas')

,provincia,vacunas_aplicadas
2,4,6
0,2,4
1,3,4
